In [1]:
# Instalar as bibliotecas pandas e scikit-learn, caso ainda não estejam instaladas
# !pip install pandas
# !pip install scikit-learn

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

# Carregar os dados
new_df_argentina_total = pd.read_csv('new_df_argentina_ml.csv')
# Reduzir o tamanho do dataset para 10% do original de forma aleatória para facilitar o processamento
new_df_argentina = new_df_argentina_total.sample(frac=0.9)

# Selecionar apenas as colunas de interesse
df = new_df_argentina[['Cidade', 'ValorFOBUS', 'PesoLiquido', 'CodigoSH4']]

# Excluir os alvos (targets) cujo número possa atrapalhar a validação cruzada com 5 folds
df = df[df.groupby('Cidade')['Cidade'].transform('count') > 5].copy()

# Dividir o dataframe em partes com 'Cidade' faltante e não faltante
df_missing = df[df['Cidade'].isna()]
df_not_missing = df.dropna(subset=['Cidade'])

# Preparar os dados para treinamento
X = df_not_missing[['ValorFOBUS', 'PesoLiquido', 'CodigoSH4']]
y = df_not_missing['Cidade']

# Verificar se há valores infinitos e substituí-los por NaN para facilitar o tratamento posterior
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Inicializar o SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Verificar se X agora contém NaN (devido à substituição ou a qualquer valor faltante anterior) e, se sim, tratar esses valores
if X.isnull().sum().sum() > 0:
    X = imputer.fit_transform(X)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Definir o modelo de Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Realizar a validação cruzada
scores = cross_val_score(clf, X, y, cv=5)

# Imprimir os resultados da validação cruzada
print("Acurácia da validação cruzada para Random Forest: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


/var/folders/fm/pz8nvy810xjdk3ywmvl5_0wr0000gn/T/ipykernel_36184/2549843179.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)


Acurácia da validação cruzada para Random Forest: 0.95 (+/- 0.01)


In [3]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Realizar a validação cruzada para obter previsões
y_pred = cross_val_predict(clf, X, y, cv=5)

# Calcular precisão e recall
precision = precision_score(y, y_pred, average='macro')  # ou 'micro', 'weighted' dependendo da sua necessidade
recall = recall_score(y, y_pred, average='macro')  # ou 'micro', 'weighted'

print("Precisão: %0.2f" % precision)
print("Recall: %0.2f" % recall)

Precisão: 0.90
Recall: 0.85


/Users/rafaeldias/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Ajustar o modelo com o conjunto de dados completo sem 'Cidade' faltante
clf.fit(X, y)



In [39]:
# Carregar os dados
new_df_argentina_total = pd.read_csv('new_df_argentina_ml_1.csv')
# Caso seja necessário, amostrar um percentual dos dados poderia ser feito assim:
# new_df_argentina_total = new_df_argentina_total1.sample(frac=0.03)

# Preparar os dados em que 'Cidade' está ausente para a previsão
X_missing_total = new_df_argentina_total[new_df_argentina_total['Cidade'].isna()][['ValorFOBUS', 'PesoLiquido', 'CodigoSH4']]
# Substituir valores infinitos por NaN para evitar erros no processamento
X_missing_total.replace([np.inf, -np.inf], np.nan, inplace=True)

# Preencher valores ausentes em X_missing_total, caso existam
if X_missing_total.isnull().sum().sum() > 0:
    # Presume-se que 'imputer' seja previamente ajustado com os dados apropriados
    X_missing_total = imputer.transform(X_missing_total)

# Prever 'Cidade' para essas linhas usando o classificador 'clf'
#  'clf' previamente treinado e adequado para a tarefa
predicted_cities_total = clf.predict(X_missing_total)

# Preencher os valores ausentes de 'Cidade' no DataFrame original
# Assim se mantenha a integridade dos dados e se complete a informação faltante
new_df_argentina_total.loc[new_df_argentina_total['Cidade'].isna(), 'Cidade'] = predicted_cities_total


In [40]:
# Verificar a quantidade de valores faltantes na coluna 'Cidade'
missing_values_after = new_df_argentina_total['Cidade'].isna().sum()

# Imprimir a quantidade de valores faltantes
print(f"Quantidade de valores faltantes em 'Cidade' após a previsão: {missing_values_after}")


Quantidade de valores faltantes em 'Cidade' após a previsão: 0


In [41]:
# Salvar o DataFrame em um arquivo CSV para ser usado nas proximas análises
new_df_argentina_total.to_csv('new_df_argentina_total_with_predictions.csv', index=False)

